In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import torch
print(torch.cuda.is_available()) # True
print(torch.cuda.device_count()) # 1
print(torch.cuda.current_device()) # 0
print(torch.cuda.device(0)) # <torch.cuda.device at 0x7efce0b03be0>
print(torch.cuda.get_device_name(0)) # 'GeForce GTX 950M'

True
1
0
NVIDIA TITAN Xp


In [4]:
import sys
sys.path.append("../../..")

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
from librep.datasets.har.loaders import (
    KuHar_BalancedView20HzMotionSenseEquivalent
)

2022-11-24 13:27:23.114247: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-24 13:27:24.670726: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-24 13:27:24.670858: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-11-24 13:27:24.670874: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [7]:
from librep.transforms.topo_ae import (
    TopologicalDimensionalityReduction,
    CustomTopoDimRedTransform
)
from librep.transforms import UMAP
from sklearn.manifold import TSNE
from experiments.Topological_ae.Experiment_utils import *
from librep.datasets.multimodal import TransformMultiModalDataset
from librep.transforms.fft import FFT

In [8]:
# MotionSense Loader
loader = KuHar_BalancedView20HzMotionSenseEquivalent(
    root_dir="../../../data/views/KuHar/balanced_20Hz_motionsense_equivalent-v1", 
    download=False
)

# Print the readme (optional)
# loader.print_readme()

In [9]:
# Load the dataset
# If concat_train_validation is true, return a tuple (train+validation, test)
train_val, test = loader.load(concat_train_validation=True, label=loader.standard_label)
train_val, test

(PandasMultiModalDataset: samples=3114, features=360, no. window=6, label_columns='standard activity code',
 PandasMultiModalDataset: samples=246, features=360, no. window=6, label_columns='standard activity code')

In [10]:
train_HD = np.array(train_val[:][0])
train_Y = np.array(train_val[:][1])
test_HD = np.array(test[:][0])
test_Y = np.array(test[:][1])

In [11]:
print(train_HD.shape, train_Y.shape, test_HD.shape, test_Y.shape)

(3114, 360) (3114,) (246, 360) (246,)


# Aplicar FFT

In [12]:
fft_transform = FFT(centered = True)
transformer = TransformMultiModalDataset(
    transforms=[fft_transform],
    new_window_name_prefix="fft."
)

In [13]:
train_dataset_fft = transformer(train_val)
test_dataset_fft = transformer(test)

In [14]:
print(train_dataset_fft.X.shape)
print(test_dataset_fft.X.shape)

(3114, 180)
(246, 180)


In [15]:
print(np.max(train_dataset_fft.X))

1199.4899017510002


In [16]:
train_HD = train_dataset_fft.X
train_LD = None
train_Y = train_dataset_fft.y
test_HD = test_dataset_fft.X
test_LD = None
test_Y = test_dataset_fft.y

In [17]:
print(train_HD.shape, train_Y.shape, test_HD.shape, test_Y.shape)

(3114, 180) (3114,) (246, 180) (246,)


# Visualization helper

In [18]:
# 0: sit (569 train, 101 validation, 170 test)
# 1: stand (569 train, 101 validation, 170 test)
# 2: walk (569 train, 101 validation, 170 test)
# 3: stair up (569 train, 101 validation, 170 test)
# 4: stair down (569 train, 101 validation, 170 test)
# 5: run (569 train, 101 validation, 170 test)
def visualize(X, Y):
    labels = ['sit', 'stand', 'walk', 'stair up', 'stair down', 'run']
    df = pd.DataFrame(dict(x=X[:,0], y=X[:,1], label=Y))
    groups = df.groupby('label')
    fig, ax = plt.subplots()
    ax.margins(0.05)
    for name, group in groups:
        ax.plot(group.x, group.y, marker='.', linestyle='', ms=8, label=labels[name])
    # Shrink current axis by 20%
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

    # Put a legend to the right of the current axis
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()

In [19]:
# Variables to reuse
model_dim = 10
model_epc = 1500

# Reducing with Convolutional Topological Autoencoders (L=0)

In [20]:
# model_lam = 1
# model_dim = 10
# model_epc = 1500
# topo_reducer = TopologicalDimensionalityReduction(
#     ae_model='ConvolutionalAutoencoder_custom_dim2',
#     lam = model_lam,
#     ae_kwargs = {'input_dims':(1, 180), 'custom_dim':model_dim},
#     input_shape = (-1, 1, 1, 180),
#     patience = None,
#     num_epochs = model_epc
# )

In [21]:
model_lam = 0
topo_reducer = CustomTopoDimRedTransform(
    model_name='ConvolutionalAutoencoder_custom_dim3',
    model_lambda=model_lam,
    patience=None,
    num_epochs=model_epc,
    from_dim=180,
    to_dim=model_dim
)

Topologically Regularized ConvolutionalAutoencoder_custom_dim3
Using python to compute signatures
ConvAECustomDim, Input: (1, 180) Inner dim: 10
ENCODER STRUCT torch.Size([4, 8, 5])


In [22]:
title_plot = "MotionSense 20Hz\nTopoAE lambda {}".format(model_lam)
topo_reducer.fit(train_HD, train_Y, title_plot=title_plot)

/home/darlinne.soto/librep-hiaac/experiments/Topological_ae/KuHar20Hz/../../../librep/transforms/topo_ae.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  in_tensor = torch.tensor(reshaped_data, device=cuda0).float()
/home/darlinne.soto/librep-hiaac/experiments/Topological_ae/KuHar20Hz/../../../librep/transforms/topo_ae.py:87: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  in_tensor = torch.tensor(reshaped_data, device=cuda0).float()


Epoch:1, P:None, Loss:492.0904, Loss-ae:492.0904, Loss-topo:279.3173
Epoch:2, P:None, Loss:411.2936, Loss-ae:411.2936, Loss-topo:274.0433
Epoch:3, P:None, Loss:384.3358, Loss-ae:384.3358, Loss-topo:331.1529
Epoch:4, P:None, Loss:370.8838, Loss-ae:370.8838, Loss-topo:620.0950
Epoch:5, P:None, Loss:301.7717, Loss-ae:301.7717, Loss-topo:1637.4410
Epoch:6, P:None, Loss:253.7304, Loss-ae:253.7304, Loss-topo:3723.6897
Epoch:7, P:None, Loss:229.8337, Loss-ae:229.8337, Loss-topo:5554.5778
Epoch:8, P:None, Loss:212.3578, Loss-ae:212.3578, Loss-topo:6905.7755
Epoch:9, P:None, Loss:200.1848, Loss-ae:200.1848, Loss-topo:7849.5191
Epoch:10, P:None, Loss:185.0492, Loss-ae:185.0492, Loss-topo:11288.9084
Epoch:11, P:None, Loss:170.5841, Loss-ae:170.5841, Loss-topo:13169.5381
Epoch:12, P:None, Loss:161.2224, Loss-ae:161.2224, Loss-topo:16640.8980
Epoch:13, P:None, Loss:154.7648, Loss-ae:154.7648, Loss-topo:18834.2260
Epoch:14, P:None, Loss:149.3900, Loss-ae:149.3900, Loss-topo:19699.9762
Epoch:15, P:No

Epoch:117, P:None, Loss:69.6500, Loss-ae:69.6500, Loss-topo:97975.7156
Epoch:118, P:None, Loss:68.8206, Loss-ae:68.8206, Loss-topo:91599.0203
Epoch:119, P:None, Loss:68.9461, Loss-ae:68.9461, Loss-topo:94375.9375
Epoch:120, P:None, Loss:69.3616, Loss-ae:69.3616, Loss-topo:94346.7375
Epoch:121, P:None, Loss:69.1478, Loss-ae:69.1478, Loss-topo:93658.8047
Epoch:122, P:None, Loss:72.2385, Loss-ae:72.2385, Loss-topo:104975.4047
Epoch:123, P:None, Loss:69.7558, Loss-ae:69.7558, Loss-topo:88644.3844
Epoch:124, P:None, Loss:69.7637, Loss-ae:69.7637, Loss-topo:99373.1391
Epoch:125, P:None, Loss:67.8613, Loss-ae:67.8613, Loss-topo:95007.9500
Epoch:126, P:None, Loss:68.0685, Loss-ae:68.0685, Loss-topo:97462.7266
Epoch:127, P:None, Loss:69.5109, Loss-ae:69.5109, Loss-topo:94443.9250
Epoch:128, P:None, Loss:68.7243, Loss-ae:68.7243, Loss-topo:100531.7250
Epoch:129, P:None, Loss:69.0679, Loss-ae:69.0679, Loss-topo:96053.2766
Epoch:130, P:None, Loss:68.5301, Loss-ae:68.5301, Loss-topo:105322.4406
Epo

Epoch:232, P:None, Loss:65.3798, Loss-ae:65.3798, Loss-topo:111554.2266
Epoch:233, P:None, Loss:65.3365, Loss-ae:65.3365, Loss-topo:113500.3859
Epoch:234, P:None, Loss:65.8546, Loss-ae:65.8546, Loss-topo:112457.6766
Epoch:235, P:None, Loss:65.9074, Loss-ae:65.9074, Loss-topo:111311.3953
Epoch:236, P:None, Loss:65.8993, Loss-ae:65.8993, Loss-topo:114386.9703
Epoch:237, P:None, Loss:65.4541, Loss-ae:65.4541, Loss-topo:117211.0469
Epoch:238, P:None, Loss:65.7872, Loss-ae:65.7872, Loss-topo:108617.3344
Epoch:239, P:None, Loss:65.4958, Loss-ae:65.4958, Loss-topo:115821.1859
Epoch:240, P:None, Loss:65.3089, Loss-ae:65.3089, Loss-topo:110318.3188
Epoch:241, P:None, Loss:65.6418, Loss-ae:65.6418, Loss-topo:112111.5297
Epoch:242, P:None, Loss:65.0867, Loss-ae:65.0867, Loss-topo:115296.7031
Epoch:243, P:None, Loss:65.6320, Loss-ae:65.6320, Loss-topo:120093.1875
Epoch:244, P:None, Loss:65.0977, Loss-ae:65.0977, Loss-topo:119604.4734
Epoch:245, P:None, Loss:66.0088, Loss-ae:66.0088, Loss-topo:1119

Epoch:346, P:None, Loss:65.0910, Loss-ae:65.0910, Loss-topo:124389.7672
Epoch:347, P:None, Loss:64.9846, Loss-ae:64.9846, Loss-topo:114163.0469
Epoch:348, P:None, Loss:64.8417, Loss-ae:64.8417, Loss-topo:118313.7547
Epoch:349, P:None, Loss:64.5378, Loss-ae:64.5378, Loss-topo:118052.9500
Epoch:350, P:None, Loss:65.0486, Loss-ae:65.0486, Loss-topo:123959.2937
Epoch:351, P:None, Loss:65.3947, Loss-ae:65.3947, Loss-topo:125259.6469
Epoch:352, P:None, Loss:64.9802, Loss-ae:64.9802, Loss-topo:117536.3469
Epoch:353, P:None, Loss:65.3115, Loss-ae:65.3115, Loss-topo:120357.2250
Epoch:354, P:None, Loss:65.4324, Loss-ae:65.4324, Loss-topo:119429.6406
Epoch:355, P:None, Loss:64.7345, Loss-ae:64.7345, Loss-topo:109836.0016
Epoch:356, P:None, Loss:65.4051, Loss-ae:65.4051, Loss-topo:118210.7141
Epoch:357, P:None, Loss:64.7314, Loss-ae:64.7314, Loss-topo:113109.5750
Epoch:358, P:None, Loss:64.8983, Loss-ae:64.8983, Loss-topo:125749.5469
Epoch:359, P:None, Loss:65.1122, Loss-ae:65.1122, Loss-topo:1181

...

Epoch:574, P:None, Loss:66.4063, Loss-ae:66.4063, Loss-topo:117686.9656
Epoch:575, P:None, Loss:67.0499, Loss-ae:67.0499, Loss-topo:119993.6687
Epoch:576, P:None, Loss:65.6819, Loss-ae:65.6819, Loss-topo:117365.7250
Epoch:577, P:None, Loss:66.5720, Loss-ae:66.5720, Loss-topo:114849.5578
Epoch:578, P:None, Loss:65.6985, Loss-ae:65.6985, Loss-topo:120676.9266
Epoch:579, P:None, Loss:66.3005, Loss-ae:66.3005, Loss-topo:115166.5000
Epoch:580, P:None, Loss:66.4565, Loss-ae:66.4565, Loss-topo:115405.3000
Epoch:581, P:None, Loss:65.8278, Loss-ae:65.8278, Loss-topo:123541.4359
Epoch:582, P:None, Loss:66.5203, Loss-ae:66.5203, Loss-topo:114510.3078
Epoch:583, P:None, Loss:66.5898, Loss-ae:66.5898, Loss-topo:114926.8844
Epoch:584, P:None, Loss:66.0603, Loss-ae:66.0603, Loss-topo:110472.4906
Epoch:585, P:None, Loss:65.6074, Loss-ae:65.6074, Loss-topo:118702.7859
Epoch:586, P:None, Loss:66.0838, Loss-ae:66.0838, Loss-topo:120829.6250
Epoch:587, P:None, Loss:66.1270, Loss-ae:66.1270, Loss-topo:1212

Epoch:688, P:None, Loss:67.3685, Loss-ae:67.3685, Loss-topo:119449.4703
Epoch:689, P:None, Loss:66.7628, Loss-ae:66.7628, Loss-topo:117732.1328
Epoch:690, P:None, Loss:66.3993, Loss-ae:66.3993, Loss-topo:110552.6578
Epoch:691, P:None, Loss:67.3433, Loss-ae:67.3433, Loss-topo:120418.1891
Epoch:692, P:None, Loss:66.7781, Loss-ae:66.7781, Loss-topo:113245.1141
Epoch:693, P:None, Loss:66.0097, Loss-ae:66.0097, Loss-topo:117350.1531
Epoch:694, P:None, Loss:66.6020, Loss-ae:66.6020, Loss-topo:111147.3281
Epoch:695, P:None, Loss:66.8111, Loss-ae:66.8111, Loss-topo:112810.4453
Epoch:696, P:None, Loss:66.5403, Loss-ae:66.5403, Loss-topo:111814.2328
Epoch:697, P:None, Loss:65.9157, Loss-ae:65.9157, Loss-topo:114955.4766
Epoch:698, P:None, Loss:66.1653, Loss-ae:66.1653, Loss-topo:114082.2297
Epoch:699, P:None, Loss:66.1351, Loss-ae:66.1351, Loss-topo:118659.6656
Epoch:700, P:None, Loss:66.9987, Loss-ae:66.9987, Loss-topo:109388.7406
Epoch:701, P:None, Loss:66.5448, Loss-ae:66.5448, Loss-topo:1129

Epoch:802, P:None, Loss:67.9738, Loss-ae:67.9738, Loss-topo:117825.1641
Epoch:803, P:None, Loss:66.8939, Loss-ae:66.8939, Loss-topo:110343.1969
Epoch:804, P:None, Loss:66.6901, Loss-ae:66.6901, Loss-topo:108183.0047
Epoch:805, P:None, Loss:67.6158, Loss-ae:67.6158, Loss-topo:109712.4531
Epoch:806, P:None, Loss:66.9675, Loss-ae:66.9675, Loss-topo:114170.5969
Epoch:807, P:None, Loss:67.4309, Loss-ae:67.4309, Loss-topo:112284.6125
Epoch:808, P:None, Loss:67.3943, Loss-ae:67.3943, Loss-topo:107453.8766
Epoch:809, P:None, Loss:68.7231, Loss-ae:68.7231, Loss-topo:105705.8766
Epoch:810, P:None, Loss:67.0367, Loss-ae:67.0367, Loss-topo:110033.5016
Epoch:811, P:None, Loss:67.3668, Loss-ae:67.3668, Loss-topo:110191.2828
Epoch:812, P:None, Loss:67.2632, Loss-ae:67.2632, Loss-topo:109613.5750
Epoch:813, P:None, Loss:66.9371, Loss-ae:66.9371, Loss-topo:110674.2266
Epoch:814, P:None, Loss:66.6737, Loss-ae:66.6737, Loss-topo:102765.8438
Epoch:815, P:None, Loss:68.6943, Loss-ae:68.6943, Loss-topo:1098

Epoch:916, P:None, Loss:68.2334, Loss-ae:68.2334, Loss-topo:106408.8828
Epoch:917, P:None, Loss:68.0384, Loss-ae:68.0384, Loss-topo:104523.9187
Epoch:918, P:None, Loss:68.3299, Loss-ae:68.3299, Loss-topo:103228.3125
Epoch:919, P:None, Loss:68.3542, Loss-ae:68.3542, Loss-topo:100953.7266
Epoch:920, P:None, Loss:67.8376, Loss-ae:67.8376, Loss-topo:105641.5125


KeyboardInterrupt: 

In [ ]:
train_LD = topo_reducer.transform(train_HD)
test_LD = topo_reducer.transform(test_HD)

In [ ]:
train_LD = np.reshape(topo_reducer.transform(train_HD), (-1, model_dim))
print('TRAIN LD RESHAPED', train_LD.shape)
test_LD = np.reshape(topo_reducer.transform(test_HD), (-1, model_dim))
print('TEST LD RESHAPED', test_LD.shape)

In [ ]:
experiments_result = run_experiments(train_HD, train_LD, train_Y, test_HD, test_LD, test_Y)
experiments_result

In [ ]:
# Visualizing
tsne = TSNE()
test_2D = tsne.fit_transform(test_LD)
visualize(test_2D, test_Y)

# Reducing with Topological Autoencoders (L=1)

In [ ]:
model_lam = 1
topo_reducer = CustomTopoDimRedTransform(
    model_name='ConvolutionalAutoencoder_custom_dim3',
    model_lambda=model_lam,
    patience=None,
    num_epochs=model_epc,
    from_dim=180,
    to_dim=model_dim
)

In [ ]:
title_plot = "MotionSense 20Hz\nTopoAE lambda {}".format(model_lam)
topo_reducer.fit(train_HD, train_Y, title_plot=title_plot)

In [ ]:
train_LD = topo_reducer.transform(train_HD)
test_LD = topo_reducer.transform(test_HD)

In [ ]:
train_LD = np.reshape(topo_reducer.transform(train_HD), (-1, model_dim))
print('TRAIN LD RESHAPED', train_LD.shape)
test_LD = np.reshape(topo_reducer.transform(test_HD), (-1, model_dim))
print('TEST LD RESHAPED', test_LD.shape)

In [ ]:
experiments_result = run_experiments(train_HD, train_LD, train_Y, test_HD, test_LD, test_Y)
experiments_result

In [ ]:
# Visualizing
tsne = TSNE()
test_2D = tsne.fit_transform(test_LD)
visualize(test_2D, test_Y)

# Reducing with Topological Autoencoders (L=5)

In [ ]:
model_lam = 5
topo_reducer = CustomTopoDimRedTransform(
    model_name='ConvolutionalAutoencoder_custom_dim3',
    model_lambda=model_lam,
    patience=None,
    num_epochs=model_epc,
    from_dim=180,
    to_dim=model_dim
)

In [ ]:
title_plot = "MotionSense 20Hz\nTopoAE lambda {}".format(model_lam)
topo_reducer.fit(train_HD, train_Y, title_plot=title_plot)

In [ ]:
train_LD = topo_reducer.transform(train_HD)
test_LD = topo_reducer.transform(test_HD)

In [ ]:
train_LD = np.reshape(topo_reducer.transform(train_HD), (-1, model_dim))
print('TRAIN LD RESHAPED', train_LD.shape)
test_LD = np.reshape(topo_reducer.transform(test_HD), (-1, model_dim))
print('TEST LD RESHAPED', test_LD.shape)

In [ ]:
experiments_result = run_experiments(train_HD, train_LD, train_Y, test_HD, test_LD, test_Y)
experiments_result

In [ ]:
# Visualizing
tsne = TSNE()
test_2D = tsne.fit_transform(test_LD)
visualize(test_2D, test_Y)

# Reducing with Topological Autoencoders (L=10)

In [ ]:
model_lam = 10
topo_reducer = CustomTopoDimRedTransform(
    model_name='ConvolutionalAutoencoder_custom_dim3',
    model_lambda=model_lam,
    patience=None,
    num_epochs=model_epc,
    from_dim=180,
    to_dim=model_dim
)

In [ ]:
title_plot = "MotionSense 20Hz\nTopoAE lambda {}".format(model_lam)
topo_reducer.fit(train_HD, train_Y, title_plot=title_plot)

In [ ]:
train_LD = topo_reducer.transform(train_HD)
test_LD = topo_reducer.transform(test_HD)

In [ ]:
train_LD = np.reshape(topo_reducer.transform(train_HD), (-1, model_dim))
print('TRAIN LD RESHAPED', train_LD.shape)
test_LD = np.reshape(topo_reducer.transform(test_HD), (-1, model_dim))
print('TEST LD RESHAPED', test_LD.shape)

In [ ]:
experiments_result = run_experiments(train_HD, train_LD, train_Y, test_HD, test_LD, test_Y)
experiments_result

In [ ]:
# Visualizing
tsne = TSNE()
test_2D = tsne.fit_transform(test_LD)
visualize(test_2D, test_Y)

# Reducing with Topological Autoencoders (L=100)

In [ ]:
model_lam = 100
topo_reducer = CustomTopoDimRedTransform(
    model_name='ConvolutionalAutoencoder_custom_dim3',
    model_lambda=model_lam,
    patience=None,
    num_epochs=model_epc,
    from_dim=180,
    to_dim=model_dim
)

In [ ]:
title_plot = "MotionSense 20Hz\nTopoAE lambda {}".format(model_lam)
topo_reducer.fit(train_HD, train_Y, title_plot=title_plot)

In [ ]:
train_LD = topo_reducer.transform(train_HD)
test_LD = topo_reducer.transform(test_HD)

In [ ]:
train_LD = np.reshape(topo_reducer.transform(train_HD), (-1, model_dim))
print('TRAIN LD RESHAPED', train_LD.shape)
test_LD = np.reshape(topo_reducer.transform(test_HD), (-1, model_dim))
print('TEST LD RESHAPED', test_LD.shape)

In [ ]:
experiments_result = run_experiments(train_HD, train_LD, train_Y, test_HD, test_LD, test_Y)
experiments_result

In [ ]:
# Visualizing
tsne = TSNE()
test_2D = tsne.fit_transform(test_LD)
visualize(test_2D, test_Y)

# Reducing with Topological Autoencoders (L=0.1)

In [ ]:
model_lam = 0.1
topo_reducer = CustomTopoDimRedTransform(
    model_name='ConvolutionalAutoencoder_custom_dim3',
    model_lambda=model_lam,
    patience=None,
    num_epochs=model_epc,
    from_dim=180,
    to_dim=model_dim
)

In [ ]:
title_plot = "MotionSense 20Hz\nTopoAE lambda {}".format(model_lam)
topo_reducer.fit(train_HD, train_Y, title_plot=title_plot)

In [ ]:
train_LD = topo_reducer.transform(train_HD)
test_LD = topo_reducer.transform(test_HD)

In [ ]:
train_LD = np.reshape(topo_reducer.transform(train_HD), (-1, model_dim))
print('TRAIN LD RESHAPED', train_LD.shape)
test_LD = np.reshape(topo_reducer.transform(test_HD), (-1, model_dim))
print('TEST LD RESHAPED', test_LD.shape)

In [ ]:
experiments_result = run_experiments(train_HD, train_LD, train_Y, test_HD, test_LD, test_Y)
experiments_result

In [ ]:
# Visualizing
tsne = TSNE()
test_2D = tsne.fit_transform(test_LD)
visualize(test_2D, test_Y)

# Reducing with Topological Autoencoders (L=0.01)

In [ ]:
model_lam = 0.01
topo_reducer = CustomTopoDimRedTransform(
    model_name='ConvolutionalAutoencoder_custom_dim3',
    model_lambda=model_lam,
    patience=None,
    num_epochs=model_epc,
    from_dim=180,
    to_dim=model_dim
)

In [ ]:
title_plot = "MotionSense 20Hz\nTopoAE lambda {}".format(model_lam)
topo_reducer.fit(train_HD, train_Y, title_plot=title_plot)

In [ ]:
train_LD = topo_reducer.transform(train_HD)
test_LD = topo_reducer.transform(test_HD)

In [ ]:
train_LD = np.reshape(topo_reducer.transform(train_HD), (-1, model_dim))
print('TRAIN LD RESHAPED', train_LD.shape)
test_LD = np.reshape(topo_reducer.transform(test_HD), (-1, model_dim))
print('TEST LD RESHAPED', test_LD.shape)

In [ ]:
experiments_result = run_experiments(train_HD, train_LD, train_Y, test_HD, test_LD, test_Y)
experiments_result

In [ ]:
# Visualizing
tsne = TSNE()
test_2D = tsne.fit_transform(test_LD)
visualize(test_2D, test_Y)